In [1]:
from kubernetes import client, config, watch
import uuid

# config.kube_config.load_kube_config("minikube_config.yml")
config.kube_config.load_kube_config("config.yml")
core_v1 = client.CoreV1Api()  #用于操作核心v1 API组中的资源，如Pod、Service、Node、Namespace等
batch_v1 = client.BatchV1Api()  #用于操作Job等资源
apps_v1 = client.AppsV1Api()  #用于操作Deployment、DaemonSet、ReplicaSet和StatefulSet
custom_api = client.CustomObjectsApi()  #用于操作CRD

In [ ]:
nginx_pod_name = 'nginx-{}'.format(uuid.uuid4())
nginx_pod = {
    'apiVersion': 'v1',
    'kind': 'Pod',
    'metadata': {
        'name': nginx_pod_name,
        'labels': {
            "app": "nginx"
        }
    },
    'spec': {
        'containers': [
            {
                'name': 'my-container',
                'image': 'nginx:1.7.9',
                "ports": [
                    {
                        "containerPort": 80
                    }
                ]
            }
        ],
        #指定调度节点
        # 'nodeSelector': {
        #     'kubernetes.io/hostname': 'bigdata4'
        # }
    }
}
#创建
core_v1.create_namespaced_pod(body=nginx_pod, namespace="default")
#监控
close = True
try:
    w = watch.Watch()
    # 监控default命名空间中的pod
    for event in w.stream(core_v1.list_namespaced_pod, namespace="default"):
        print("event: %s %s" % (event['type'], event['object'].metadata.name))
        if event['object'].metadata.name == nginx_pod_name and event['type'] == "DELETED":
            print("pod:{} life end,exit watch".format(nginx_pod_name))
            w.stop()
            close = False
            break
except Exception as e:
    e.with_traceback()
finally:
    if close:
        core_v1.delete_namespaced_pod(name=nginx_pod_name, namespace="default")

event: ADDED nginx-deploy-57456c49b5-rn8mm
event: ADDED nginx-deploy-57456c49b5-t827s
event: ADDED nginx-d24303ae-92a8-4da2-9fbb-31bd3b61cd03
event: ADDED nginx-2fa08c5f-1436-4a30-a3c1-627f3b912022
event: MODIFIED nginx-2fa08c5f-1436-4a30-a3c1-627f3b912022
event: MODIFIED nginx-2fa08c5f-1436-4a30-a3c1-627f3b912022
event: MODIFIED nginx-2fa08c5f-1436-4a30-a3c1-627f3b912022


In [ ]:
# 监控具有特定标签的pod
w = watch.Watch()
label_selector = "k8s.kuboard.cn/name=nginx-deploy"
for event in w.stream(core_v1.list_namespaced_pod, namespace="default", label_selector=label_selector):
    pod = event['object']
    if event['type'] == 'ADDED':
        print("Pod Added: %s" % pod.metadata.name)
    elif event['type'] == 'MODIFIED':
        print("Pod Modified: %s" % pod.metadata.name)
    elif event['type'] == 'DELETED':
        print("Pod Deleted: %s" % pod.metadata.name)
    elif event['type'] == 'ERROR':
        print("Error encountered: %s" % event['object'])


Pod Added: nginx-deploy-7669d4b87b-z5zl6
Pod Added: nginx-deploy-7669d4b87b-ft8cz
Pod Added: nginx-deploy-57456c49b5-t827s
Pod Modified: nginx-deploy-57456c49b5-t827s
Pod Modified: nginx-deploy-57456c49b5-t827s
Pod Modified: nginx-deploy-57456c49b5-t827s
Pod Modified: nginx-deploy-57456c49b5-t827s
Pod Modified: nginx-deploy-7669d4b87b-ft8cz
Pod Added: nginx-deploy-57456c49b5-rn8mm
Pod Modified: nginx-deploy-57456c49b5-rn8mm
Pod Modified: nginx-deploy-57456c49b5-rn8mm
Pod Modified: nginx-deploy-57456c49b5-rn8mm
Pod Modified: nginx-deploy-7669d4b87b-ft8cz
Pod Modified: nginx-deploy-57456c49b5-rn8mm
Pod Modified: nginx-deploy-7669d4b87b-z5zl6
Pod Modified: nginx-deploy-7669d4b87b-z5zl6
Pod Modified: nginx-deploy-7669d4b87b-ft8cz
Pod Deleted: nginx-deploy-7669d4b87b-ft8cz
Pod Modified: nginx-deploy-7669d4b87b-z5zl6
Pod Deleted: nginx-deploy-7669d4b87b-z5zl6


In [2]:
# 监控定义CRD下所有CR
group = "stable.example.com"
version = "v1"
plural = "crontabs"
namespace = "default"
try:
    w = watch.Watch()
    for event in w.stream(custom_api.list_namespaced_custom_object, group=group, version=version, namespace=namespace,
                          plural=plural):
        print(event)
        print("event: %s %s" % (event['type'], event["object"]["metadata"]["name"]))
        if event['type'] == "DELETED":
            w.stop()
            break
except Exception as e:
    e.with_traceback()

{'type': 'ADDED', 'object': {'apiVersion': 'stable.example.com/v1', 'kind': 'CronTab', 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"stable.example.com/v1","kind":"CronTab","metadata":{"annotations":{},"name":"my-new-cron-object","namespace":"default"},"spec":{"cronSpec":"* * * * */5","image":"my-awesome-cron-image","replicas":3}}\n'}, 'creationTimestamp': '2023-06-07T02:12:29Z', 'generation': 1, 'name': 'my-new-cron-object', 'namespace': 'default', 'resourceVersion': '7145600', 'selfLink': '/apis/stable.example.com/v1/namespaces/default/crontabs/my-new-cron-object', 'uid': '61bf5863-c484-41f0-80e6-5036355786ea'}, 'spec': {'cronSpec': '* * * * */5', 'image': 'my-awesome-cron-image', 'replicas': 3}}, 'raw_object': {'apiVersion': 'stable.example.com/v1', 'kind': 'CronTab', 'metadata': {'annotations': {'kubectl.kubernetes.io/last-applied-configuration': '{"apiVersion":"stable.example.com/v1","kind":"CronTab","metadata":{"annotations":{},"